In [1]:
import pandas as pd


facebook_spend = pd.read_csv('Facebook Spend August.csv')
shopify = pd.read_csv('Shopify Orders August_v2.csv')
indigogo = pd.read_csv('Indiegogo Orders August v2.csv')
celery = pd.read_csv('Celery Orders August v2.csv')

In [2]:
facebook_spend.head()

,Country,Reach,Frequency,Amount spent (USD),Clicks (all),CTR (all),CPC (all) (USD),Impressions,"CPM (cost per 1,000 impressions) (USD)",Cost per unique link click (USD),Unique link clicks,CTR (link click-through rate)
0,TH,12287,28.61,444.0,"2,042",0.58,$0.22,"351,592",$1.26,$0.35,"1,280",0.44
1,FR,19454,9.11,495.0,"1,803",1.02,$0.27,"177,225",$2.79,$0.77,640,0.62
2,NL,14335,8.72,427.0,"1,393",1.11,$0.31,"124,974",$3.42,$1.11,384,0.62
3,BE,6143,12.42,213.0,709,0.93,$0.30,"76,274",$2.79,$1.66,128,0.57
4,US,1992499,1.87,28181.0,"141,232",3.79,$0.20,"3,730,211",$7.55,$0.30,"95,232",2.64


In [3]:
ordered_by_ctr = facebook_spend.sort_values(by = 'CTR (all)', ascending = False)
ordered_by_ctr.head()

,Country,Reach,Frequency,Amount spent (USD),Clicks (all),CTR (all),CPC (all) (USD),Impressions,"CPM (cost per 1,000 impressions) (USD)",Cost per unique link click (USD),Unique link clicks,CTR (link click-through rate)
9,GB,506828,2.29,5276.0,"56,875",4.91,$0.09,"1,158,709",$4.55,$0.13,"40,064",3.71
13,AU,674747,2.06,8849.0,"65,180",4.69,$0.14,"1,391,017",$6.36,$0.20,"43,648",3.38
121,PW,0,-,0.0,9,4.41,$0.02,204,$0.83,NaN,NaN,4.41
4,US,1992499,1.87,28181.0,"141,232",3.79,$0.20,"3,730,211",$7.55,$0.30,"95,232",2.64
150,UG,0,-,0.0,2,3.77,$0.07,53,$2.64,NaN,NaN,1.89


In [4]:
merged_country = ordered_by_ctr.merge(shopify, how = 'inner', left_on = 'Country',right_on = 'Shipping Country')
merged_country.head()

,Country,Reach,Frequency,Amount spent (USD),Clicks (all),CTR (all),CPC (all) (USD),Impressions,"CPM (cost per 1,000 impressions) (USD)",Cost per unique link click (USD),...,Shipping Method,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Shipping Country
0,GB,506828,2.29,5276.0,"56,875",4.91,$0.09,"1,158,709",$4.55,$0.13,...,Free worldwide shipping,1,Men's Watch Strap - Gunmetal/Tan - 38MM / Gunm...,65,65.0,STR-GN-TN-38,True,True,fulfilled,GB
1,GB,506828,2.29,5276.0,"56,875",4.91,$0.09,"1,158,709",$4.55,$0.13,...,NaN,1,Men's The Classic - Gunmetal/Navy - 38mm / Gun...,249,NaN,CLA-GN-NB-38,True,True,fulfilled,GB
2,GB,506828,2.29,5276.0,"56,875",4.91,$0.09,"1,158,709",$4.55,$0.13,...,Free worldwide shipping,1,Men's The Classic - Gunmetal/Tan - 38mm / Gunm...,249,NaN,CLA-GN-TN-38,True,True,fulfilled,GB
3,GB,506828,2.29,5276.0,"56,875",4.91,$0.09,"1,158,709",$4.55,$0.13,...,Free worldwide shipping,1,Women's The Classic - Rose Gold/Tan - 34mm / R...,249,NaN,CLA-RG-TN-34,True,True,fulfilled,GB
4,GB,506828,2.29,5276.0,"56,875",4.91,$0.09,"1,158,709",$4.55,$0.13,...,Free worldwide shipping,1,Women's Watch Strap - Silver/Mocha - 38MM / Si...,65,65.0,STR-SS-MO-38,True,True,fulfilled,GB


In [8]:
country_cost = facebook_spend[['Country','Amount spent (USD)']]
country_cost['Amount spent (USD)'].astype('float',errors = 'ignore', inplace = True)
by_cost = merged_country.groupby('Country').agg({'Country':'count','Lineitem price':'sum'}).sort_values(by = 'Lineitem price',ascending = False).rename({'Country':'Count'},axis = 1).reset_index()
by_cost.head()

,Country,Count,Lineitem price
0,US,345,74594
1,SG,184,38366
2,AU,161,34940
3,CA,92,19852
4,HK,75,17321


In [15]:
country_ROI = by_cost.merge(country_cost, how = 'inner', on = 'Country')
country_ROI['ROI'] = country_ROI['Lineitem price'] / country_ROI['Amount spent (USD)']
country_ROI.sort_values(by = 'ROI',ascending = False).head(20)

,Country,Count,Lineitem price,Amount spent (USD),ROI
20,ZA,2,498,2.0,249.000000
24,NG,1,249,1.0,249.000000
21,PK,2,314,4.0,78.500000
6,NO,19,4215,144.0,29.270833
12,IN,8,1311,45.0,29.133333
25,HU,1,249,13.0,19.153846
15,NZ,4,977,61.0,16.016393
26,QA,1,249,16.0,15.562500
27,RO,1,249,23.0,10.826087
16,CN,4,812,91.0,8.923077
